In [0]:
from pyspark.sql.functions import col
import pyspark.sql.functions as F
from pyspark.sql.functions import udf
from pyspark.sql.functions import when, count, col
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [0]:
"""from elasticsearch import Elasticsearch, helpers

esURL = "10.0.0.8"

es = Elasticsearch([{'host':esURL}], timeout= 60000)

_index = "full_bus_df"

df_from_es = spark.read\
                .format("org.elasticsearch.spark.sql")\
                .option("es.nodes.wan.only","true")\
                .option("es.port","9200")\
                .option("es.nodes",esURL)\
                .option("es.nodes.client.only", "false")\
                .option("pushdown", "true")\
                .option("es.scroll.keepalive", "120m")\
                .load(_index)

display(df_from_es)

In [0]:
df = spark.read.json('/mnt/dacoursedatabricksstg/dacoursedatabricksdata/busFile')

In [0]:
df = df.drop("direction", "poiId", "poiId2", "probability", "loc")\
               .withColumn("id", col("_id.$oid"))\
               .withColumn("calendar", col("calendar.$numberLong").cast("bigint"))\
               .withColumn("timestamp", col("timestamp.$numberLong").cast("bigint"))\
               .drop("_id")
df_transform = df.withColumn('_timestamp', df['timestamp'].cast('bigint'))
df_transform = df_transform.withColumn('dateTime', F.to_timestamp((df_transform['_timestamp'] / 1000)))


def part_of_day(ts_str):
  hour = int(str(ts_str)[11:13])
  if hour >4 :
    if hour <6:
      return 'Dawn'
    if hour < 9:
      return 'Morning'
    if hour < 12:
      return 'Mid-Morning'
    if hour < 14:
      return 'Noon'
    if hour < 16:
      return 'After-Noon'
    if hour < 21:
      return 'Evening'
    if hour <24:
      return 'Night'
  else:
    if hour < 1:
      return 'Night'
    if hour <=4 :
      return 'Wee Hours'
    
part_of_day_udf = udf(part_of_day)

df_transform = df_transform.select('id','delay','congestion','vehicleSpeed','distanceCovered','angle','dateType',\
                                   'lineID','journeyPatternId','vehicleID','busStop','areaID','areaID2','areaID3','gridID','latitude','longitude',\
                                   'dateTime',part_of_day_udf("dateTime").alias("partOfDay")).withColumnRenamed('dateType','WeekendOrNot')

hour_extract = udf(lambda x: int(str(x)[11:13]))

df_transform = df_transform.withColumn("currentHour",hour_extract('dateTime').cast("int")).withColumn("congestion", col("congestion").cast("boolean")).withColumn("congestion", col("congestion").cast("integer"))

In [0]:
display(df_transform)

id,delay,congestion,vehicleSpeed,distanceCovered,angle,WeekendOrNot,lineID,journeyPatternId,vehicleID,busStop,areaID,areaID2,areaID3,gridID,latitude,longitude,dateTime,partOfDay,currentHour
595a2300e45b4b2ea81ae778,-2,0,0,0.22027568625824304,45.0,1,27,00271001,44087,2377,15001,234,3750,"132,86",53.315958,-6.334248,2017-07-03T10:56:37.000+0000,Mid-Morning,10
595a2300e45b4b2ea81ae779,-104,0,0,0.16586637987086802,225.0,1,18,00181001,33467,2485,15138,236,3784,"146,88",53.319117,-6.290017,2017-07-03T10:56:37.000+0000,Mid-Morning,10
595a2300e45b4b2ea81ae77a,124,0,0,0.048167532049173775,225.0,1,13,00130002,43041,113,18342,286,4585,"155,131",53.395714,-6.26408,2017-07-03T10:56:37.000+0000,Mid-Morning,10
595a2300e45b4b2ea81ae77b,-147,0,0,0.23610840881390818,225.0,1,25,025B0002,33401,7239,3062,191,3062,"120,107",53.353845,-6.367692,2017-07-03T10:56:37.000+0000,Mid-Morning,10
595a2300e45b4b2ea81ae77c,-423,0,0,0.0758441143128766,45.0,1,18,00180001,33464,779,15536,242,3883,"160,93",53.327876,-6.248047,2017-07-03T10:56:37.000+0000,Mid-Morning,10
595a2300e45b4b2ea81ae77d,61,0,0,0.08946480481101295,45.0,1,11,00110001,44110,404,73060,285,4565,"157,103",53.345593,-6.257684,2017-07-03T10:56:37.000+0000,Mid-Morning,10
595a2300e45b4b2ea81ae77e,194,0,0,0.03299085710397742,45.0,1,123,01231001,44103,1278,73060,285,4565,"157,103",53.346069,-6.257612,2017-07-03T10:56:37.000+0000,Mid-Morning,10
595a2300e45b4b2ea81ae77f,-373,0,0,0.448449642683063,45.0,1,46A,046A0001,36013,2007,15646,244,3911,"170,83",53.310553,-6.21956,2017-07-03T10:56:37.000+0000,Mid-Morning,10
595a2300e45b4b2ea81ae780,12,0,0,0.1615753977654222,45.0,1,41,00411001,33336,4906,1164,290,1164,"162,166",53.459475,-6.242159,2017-07-03T10:56:37.000+0000,Mid-Morning,10
595a2300e45b4b2ea81ae781,87,0,0,0.0,-2.0,1,123,01231001,44107,510,18309,285,4577,"161,110",53.359226,-6.245823,2017-07-03T10:56:37.000+0000,Mid-Morning,10


In [0]:
true_values_samp = df_transform.filter(col("congestion") == True)

In [0]:
false_values_samp = df_transform.filter(col("congestion") == False).sample(False, fraction = 0.011)

In [0]:
dataset = false_values_samp.union(true_values_samp)
display(dataset)

id,delay,congestion,vehicleSpeed,distanceCovered,angle,WeekendOrNot,lineID,journeyPatternId,vehicleID,busStop,areaID,areaID2,areaID3,gridID,latitude,longitude,dateTime,partOfDay,currentHour
595a2300e45b4b2ea81ae77a,124,0,0,0.048167532049173775,225.0,1,13,00130002,43041,113,18342,286,4585,"155,131",53.395714,-6.26408,2017-07-03T10:56:37.000+0000,Mid-Morning,10
595a2304e45b4b2ea81ae7e9,169,0,0,0.0,0.0,0,39,039A1001,36068,7453,71644,279,4477,"150,103",53.346443,-6.279392,2017-07-03T10:56:49.000+0000,Mid-Morning,10
595a2308e45b4b2ea81ae861,-316,0,0,0.7088765837224045,45.0,0,145,01450007,36035,760,62226,242,3888,"166,88",53.319375,-6.232309,2017-07-03T10:56:55.000+0000,Mid-Morning,10
595a2310e45b4b2ea81ae8e0,-87,0,0,0.0,0.0,0,46A,046A0001,36006,808,17884,279,4470,"145,107",53.35385,-6.295462,2017-07-03T10:56:59.000+0000,Mid-Morning,10
595a2310e45b4b2ea81ae957,0,0,0,0.0,0.0,0,150,01501001,33519,4863,3740,233,3740,"137,77",53.299175,-6.319478,2017-07-03T10:57:04.000+0000,Mid-Morning,10
595a2310e45b4b2ea81ae9c8,519,0,0,0.01484005014499464,45.0,0,84,00840003,43081,4252,256,256,256,"221,25",53.114018,-6.065892,2017-07-03T10:57:05.000+0000,Mid-Morning,10
595a2314e45b4b2ea81aea13,364,0,0,0.2606929701382312,45.0,0,11,00110001,44112,3017,3875,241,3875,"166,75",53.295368,-6.232168,2017-07-03T10:57:07.000+0000,Mid-Morning,10
595a2314e45b4b2ea81aea14,364,0,0,0.0,-2.0,0,9,00090001,44218,336,73060,285,4565,"157,103",53.346171,-6.257756,2017-07-03T10:57:07.000+0000,Mid-Morning,10
595a2320e45b4b2ea81aea8d,941,0,0,0.1501491614272069,45.0,0,42,00420002,38078,3626,1265,316,1265,"192,158",53.444358,-6.152411,2017-07-03T10:57:13.000+0000,Mid-Morning,10
595a2320e45b4b2ea81aea9a,452,0,0,0.0,-2.0,0,102,01021001,40016,3585,1265,316,1265,"191,161",53.450534,-6.154959,2017-07-03T10:57:14.000+0000,Mid-Morning,10


In [0]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler, VectorIndexer
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

features = ["areaID", "currentHour", "WeekendOrNot", "delay", "latitude", "longitude", "vehicleSpeed"]
va = VectorAssembler(inputCols=features, outputCol="features")
rf = RandomForestClassifier(labelCol="congestion", featuresCol="features")
pipeline = Pipeline(stages=[va, rf])
f1_evaluator = MulticlassClassificationEvaluator(labelCol="congestion", predictionCol="prediction", metricName="f1")

In [0]:
train_set_small, test_set_small = dataset.randomSplit([0.8, 0.2])
model = pipeline.fit(train_set_small)
predictions = model.transform(test_set_small)
display(predictions)

In [0]:
f1 = f1_evaluator.evaluate(predictions)
recall = predictions.filter((col("congestion") == 1) & (col("prediction") == 1)).count() / predictions.filter(col("congestion") == 1).count()
print(f"f1: {f1}, recall: {recall}")

In [0]:
dataset_unc_filtered = dataset.filter(col("delay") <= 900).filter(col("delay")>-900).filter(col("vehicleSpeed") <= -1).filter(col("vehicleSpeed")>130)

In [0]:
train_set, test_set = dataset_unc_filtered.randomSplit([0.8, 0.2])
model = pipeline.fit(train_set)
predictions = model.transform(test_set)
display(predictions)

id,delay,congestion,vehicleSpeed,distanceCovered,angle,WeekendOrNot,lineID,journeyPatternId,vehicleID,busStop,areaID,areaID2,areaID3,gridID,latitude,longitude,dateTime,partOfDay,currentHour,features,rawPrediction,probability,prediction
597510dfe45b4b5d503f1f20,141,0,0,0.16516981395284672,45.0,0,25,025A1002,33609,4619,2935,183,2935,"101,100",53,-6,2017-07-23T21:10:29.000+0000,Night,21,"List(1, 7, List(), List(2935.0, 21.0, 0.0, 141.0, 53.0, -6.0, 0.0))","List(1, 2, List(), List(4.818431489446669, 15.18156851055333))","List(1, 2, List(), List(0.24092157447233348, 0.7590784255276665))",1.0
5a637ce4e45b4b35fc95c52f,-224,0,7,0.03817019494611823,225.0,1,67,00670001,43149,3008,150,150,150,"48,118",53,-6,2018-01-20T17:31:02.000+0000,Evening,17,"List(1, 7, List(), List(150.0, 17.0, 1.0, -224.0, 53.0, -6.0, 7.0))","List(1, 2, List(), List(17.13097228442056, 2.8690277155794397))","List(1, 2, List(), List(0.8565486142210281, 0.14345138577897198))",0.0
5a6f1d49e45b4b35fc30e12b,291,0,0,0.0,-2.0,0,11,00111001,44115,3028,974,243,974,"167,73",53,-6,2018-01-29T13:10:05.000+0000,Noon,13,"List(1, 7, List(), List(974.0, 13.0, 0.0, 291.0, 53.0, -6.0, 0.0))","List(1, 2, List(), List(2.805789395326381, 17.19421060467362))","List(1, 2, List(), List(0.140289469766319, 0.8597105302336809))",1.0
5a90556ce45b4b77085801da,530,0,0,0.0,-2.0,0,15,00151003,44216,497,18275,285,4568,"159,106",53,-6,2018-02-23T17:54:48.000+0000,Evening,17,"List(1, 7, List(), List(18275.0, 17.0, 0.0, 530.0, 53.0, -6.0, 0.0))","List(1, 2, List(), List(1.5337642749407456, 18.466235725059256))","List(1, 2, List(), List(0.07668821374703727, 0.9233117862529626))",1.0
5b1ee017f0e26f6314cddbb1,-576,0,32,0.1911939144033726,225.0,0,7,007A0002,43099,3207,4285,267,4285,"196,65",53,-6,2018-06-11T20:48:10.000+0000,Evening,20,"List(1, 7, List(), List(4285.0, 20.0, 0.0, -576.0, 53.0, -6.0, 32.0))","List(1, 2, List(), List(17.90933660933661, 2.0906633906633907))","List(1, 2, List(), List(0.8954668304668305, 0.10453316953316953))",0.0
595e8a30e45b4b64f35a0ca9,389,0,0,0.026020467397116727,45.0,0,46A,046A0001,36009,4567,17121,267,4280,"194,68",53,-6,2017-07-06T19:06:00.000+0000,Evening,19,"List(1, 7, List(), List(17121.0, 19.0, 0.0, 389.0, 53.0, -6.0, 0.0))","List(1, 2, List(), List(5.895520905990855, 14.104479094009145))","List(1, 2, List(), List(0.29477604529954277, 0.7052239547004573))",1.0
5a0743c4e45b4b356e2eabd5,-122,0,80,0.35656557863670924,225.0,1,38,038A0001,33585,401,73057,285,4565,"157,101",53,-6,2017-11-11T18:38:30.000+0000,Evening,18,"List(1, 7, List(), List(73057.0, 18.0, 1.0, -122.0, 53.0, -6.0, 80.0))","List(1, 2, List(), List(18.28333333333333, 1.7166666666666668))","List(1, 2, List(), List(0.9141666666666666, 0.08583333333333334))",0.0
5af467c2f0e26f7d110ea8c5,394,0,10,0.06098761672356259,225.0,0,83,00831001,43041,1355,15188,236,3796,"154,99",53,-6,2018-05-10T15:39:39.000+0000,After-Noon,15,"List(1, 7, List(), List(15188.0, 15.0, 0.0, 394.0, 53.0, -6.0, 10.0))","List(1, 2, List(), List(14.728239928239926, 5.271760071760071))","List(1, 2, List(), List(0.7364119964119964, 0.2635880035880036))",0.0
5b26819df0e26f631438ef24,142,0,31,0.17505064326942804,225.0,1,18,00181001,33638,2655,3069,191,3069,"127,101",53,-6,2018-06-17T15:43:07.000+0000,After-Noon,15,"List(1, 7, List(), List(3069.0, 15.0, 1.0, 142.0, 53.0, -6.0, 31.0))","List(1, 2, List(), List(16.218129571577848, 3.7818704284221525))","List(1, 2, List(), List(0.8109064785788924, 0.18909352142110764))",0.0
5a2d68a0e45b4b179ccb48cf,-12,0,61,0.3394601372351986,45.0,1,67,00671002,44296,3936,157,157,157,"69,100",53,-6,2017-12-10T17:01:48.000+0000,Evening,17,"List(1, 7, List(), List(157.0, 17.0, 1.0, -12.0, 53.0, -6.0, 61.0))","List(1, 2, List(), List(17.24843260188088, 2.751567398119122))","List(1, 2, List(), List(0.8624216300940439, 0.13757836990595612))",0.0


In [0]:
f1 = f1_evaluator.evaluate(predictions)
recall = predictions.filter((col("congestion") == 1) & (col("prediction") == 1)).count() / predictions.filter(col("congestion") == 1).count()
print(f"f1: {f1}, recall: {recall}")

f1: 0.8040752351097178, recall: 1.0

In [0]:
from pyspark.sql.functions import lit

median_delay = df_filtered.approxQuantile("delay", [0.5], 0)[0]
median_speed = df_filtered.approxQuantile("speed", [0.5], 0)[0]


def _impute_med_delay(delay, median):
  if -900<delay and delay<=900:
    return float(delay)
  else:
    return float(median)
  
def _impute_med_speed(speed, median):
  if -1<speed and speed<=130:
    return float(speed)
  else:
    return float(median)

  
impute_med_delay = udf(_impute_med_delay)
impute_med_speed = udf(_impute_med_speed)

med_imputed = dataset.withColumn("median_delay", lit(float(median_delay))).withColumn("delay", impute_med("delay","median_delay").cast("float")).df.withColumn("median_speed", lit(float(median_speed))).withColumn("speed", impute_med("speed","median_speed").cast("float"))

display(med_imputed)

In [0]:
train_set, test_set = med_imputed.randomSplit([0.8, 0.2])
model = pipeline.fit(train_set)
predictions = model.transform(test_set)
display(predictions)

id,delay,congestion,vehicleSpeed,distanceCovered,angle,WeekendOrNot,lineID,journeyPatternId,vehicleID,busStop,areaID,areaID2,areaID3,gridID,latitude,longitude,dateTime,partOfDay,currentHour,median_delay,features,rawPrediction,probability,prediction
5a637ce4e45b4b35fc95c52f,-224.0,0,7,0.03817019494611823,225.0,1,67,00670001,43149,3008,150,150,150,"48,118",53,-6,2018-01-20T17:31:02.000+0000,Evening,17,141.0,"List(1, 7, List(), List(150.0, 17.0, 1.0, -224.0, 53.0, -6.0, 7.0))","List(1, 2, List(), List(15.928571428571429, 4.071428571428571))","List(1, 2, List(), List(0.7964285714285715, 0.20357142857142857))",0.0
5a34dcfee45b4b179c33dbf9,-14.0,0,56,0.3137637777767469,45.0,1,67,00671002,44006,3939,637,159,637,"73,102",53,-6,2017-12-16T08:44:03.000+0000,Morning,8,141.0,"List(1, 7, List(), List(637.0, 8.0, 1.0, -14.0, 53.0, -6.0, 56.0))","List(1, 2, List(), List(17.761904761904763, 2.238095238095238))","List(1, 2, List(), List(0.8880952380952382, 0.11190476190476191))",0.0
5acf8317e45b4b5d971eb53d,45.0,0,0,0.0,-2.0,0,31,00320002,38075,523,18323,285,4580,"166,113",53,-6,2018-04-12T16:02:27.000+0000,Evening,16,141.0,"List(1, 7, List(), List(18323.0, 16.0, 0.0, 45.0, 53.0, -6.0, 0.0))","List(1, 2, List(), List(7.679270216771468, 12.320729783228533))","List(1, 2, List(), List(0.3839635108385734, 0.6160364891614266))",1.0
5a31714ce45b4b179c085f12,162.0,0,3,0.03354622160082185,45.0,0,123,01231001,44191,335,73060,285,4565,"157,103",53,-6,2017-12-13T18:27:57.000+0000,Evening,18,141.0,"List(1, 7, List(), List(73060.0, 18.0, 0.0, 162.0, 53.0, -6.0, 3.0))","List(1, 2, List(), List(11.483340437624323, 8.516659562375676))","List(1, 2, List(), List(0.5741670218812162, 0.4258329781187838))",0.0
5a71bc0ce45b4b35fc5ac516,55.0,0,0,0.0030017230372346992,45.0,0,747,07470002,40042,2001,71651,279,4477,"151,102",53,-6,2018-01-31T12:51:55.000+0000,Noon,12,141.0,"List(1, 7, List(), List(71651.0, 12.0, 0.0, 55.0, 53.0, -6.0, 0.0))","List(1, 2, List(), List(8.551889264390516, 11.448110735609484))","List(1, 2, List(), List(0.42759446321952577, 0.5724055367804742))",1.0
59f2e4c2e45b4b634128e23c,-362.0,0,31,0.18323165898214797,225.0,0,757,07571002,40045,7398,18299,285,4574,"163,104",53,-6,2017-10-27T07:47:23.000+0000,Morning,7,141.0,"List(1, 7, List(), List(18299.0, 7.0, 0.0, -362.0, 53.0, -6.0, 31.0))","List(1, 2, List(), List(19.356954840650495, 0.6430451593495072))","List(1, 2, List(), List(0.9678477420325245, 0.03215225796747535))",0.0
5a05871ce45b4b356e147fff,141.0,0,18,0.10542146616975612,45.0,0,150,01501003,33318,2003,71675,279,4479,"153,102",53,-6,2017-11-10T11:01:29.000+0000,Mid-Morning,11,141.0,"List(1, 7, List(), List(71675.0, 11.0, 0.0, 141.0, 53.0, -6.0, 18.0))","List(1, 2, List(), List(17.358277591973245, 2.6417224080267556))","List(1, 2, List(), List(0.8679138795986623, 0.13208612040133777))",0.0
5a35672ce45b4b179c3da04b,291.0,0,6,0.03725308846065277,225.0,1,27A,027A1003,44204,7568,73072,285,4566,"157,105",53,-6,2017-12-16T18:33:43.000+0000,Evening,18,141.0,"List(1, 7, List(), List(73072.0, 18.0, 1.0, 291.0, 53.0, -6.0, 6.0))","List(1, 2, List(), List(16.58611111111111, 3.4138888888888888))","List(1, 2, List(), List(0.8293055555555556, 0.17069444444444445))",0.0
5a69e671e45b4b35fceddb31,80.0,0,28,0.4432114508859348,225.0,0,7,00071005,44230,6333,4258,265,4258,"185,77",53,-6,2018-01-25T14:14:52.000+0000,After-Noon,14,141.0,"List(1, 7, List(), List(4258.0, 14.0, 0.0, 80.0, 53.0, -6.0, 28.0))","List(1, 2, List(), List(19.291806535565748, 0.708193464434253))","List(1, 2, List(), List(0.9645903267782874, 0.03540967322171265))",0.0
5b20c9e7f0e26f6314e83d91,130.0,0,7,0.08976022484751722,45.0,0,41,00410001,44196,1626,18380,286,4594,"163,135",53,-6,2018-06-13T07:38:10.000+0000,Morning,7,141.0,"List(1, 7, List(), List(18380.0, 7.0, 0.0, 130.0, 53.0, -6.0, 7.0))","List(1, 2, List(), List(13.690288173983825, 6.309711826016174))","List(1, 2, List(), List(0.6845144086991912, 0.31548559130080867))",0.0


In [0]:
f1 = f1_evaluator.evaluate(predictions)
recall = predictions.filter((col("congestion") == 1) & (col("prediction") == 1)).count() / predictions.filter(col("congestion") == 1).count()
print(f"f1: {f1}, recall: {recall}")

f1: 0.8378378378378379, recall: 0.8571428571428571

In [0]:
mean_delay  = dataset_unc_filtered.groupBy().avg('delay').collect()[0][0]
mean_speed = dataset_unc_filtered.groupBy().avg('speed').collect()[0][0]


def _impute_mean_delay(delay, mean):
  if -900<delay and delay<=900:
    return float(delay)
  else:
    return float(mean)

def _impute_mean_speed(speed, mean):
  if -1<speed and speed<=130:
    return float(speed)
  else:
    return float(mean)
  
  
impute_mean_delay = udf(_impute_mean_delay)
impute_mean_speed = udf(_impute_mean_speed)

mean_imputed = dataset.withColumn("mean_delay", lit(float(mean_delay))).withColumn("delay", impute_med("delay","mean_delay").cast("float")).withColumn("mean_speed", lit(float(mean_speed))).withColumn("speed", impute_med("speed","mean_speed").cast("float"))
display(mean_imputed)

id,delay,congestion,vehicleSpeed,distanceCovered,angle,WeekendOrNot,lineID,journeyPatternId,vehicleID,busStop,areaID,areaID2,areaID3,gridID,latitude,longitude,dateTime,partOfDay,currentHour,mean_delay
5b965216f0e26f3d4e2a418e,52.0,0,36,0.2149286706335998,225.0,0,67,00670001,44218,7078,17889,279,4472,"146,104",53,-6,2018-09-10T11:14:24.000+0000,Mid-Morning,11,136.53424657534248
597510dfe45b4b5d503f1f20,141.0,0,0,0.16516981395284672,45.0,0,25,025A1002,33609,4619,2935,183,2935,"101,100",53,-6,2017-07-23T21:10:29.000+0000,Night,21,136.53424657534248
5a79dfe1e45b4b0b521156e3,244.0,0,0,0.0020059150470843837,45.0,0,11,00111001,44241,878,3887,242,3887,"164,81",53,-6,2018-02-06T17:03:23.000+0000,Evening,17,136.53424657534248
5ab0d786e45b4b101361147d,564.0,0,11,0.06554059480405244,225.0,0,49,00490001,33531,2532,2359,147,2359,"125,64",53,-6,2018-03-20T09:42:25.000+0000,Mid-Morning,9,136.53424657534248
5b16ce0ff0e26f276439eee9,136.53424,0,23,0.15167942567170192,225.0,0,27B,027B0001,33497,6234,4544,283,4544,"155,143",53,-6,2018-06-05T17:53:13.000+0000,Evening,17,136.53424657534248
5aec0d00f0e26f116182436c,136.53424,0,16,0.09439793136792597,225.0,0,7,007B1002,44029,776,15559,242,3889,"163,91",53,-6,2018-05-04T07:34:19.000+0000,Morning,7,136.53424657534248
5b1ee017f0e26f6314cddbb1,-576.0,0,32,0.1911939144033726,225.0,0,7,007A0002,43099,3207,4285,267,4285,"196,65",53,-6,2018-06-11T20:48:10.000+0000,Evening,20,136.53424657534248
5ae85ca6e45b4b5d974cb12b,-181.0,0,21,0.11741874248100537,45.0,0,26,066A1002,44253,1479,71676,279,4479,"153,103",53,-6,2018-05-01T12:25:05.000+0000,Noon,12,136.53424657534248
5a9eeb1ee45b4b7708cc3f9b,505.0,0,12,0.1044455544052218,45.0,0,39,039A1001,44291,1906,4451,277,4451,"142,112",53,-6,2018-03-06T19:25:11.000+0000,Evening,19,136.53424657534248
5a54d6cee45b4b563c37f8a5,147.0,0,14,0.05662943366863649,45.0,0,102,01021001,33342,6033,1163,290,1163,"164,159",53,-6,2018-01-09T14:50:40.000+0000,After-Noon,14,136.53424657534248


In [0]:
train_set, test_set = mean_imputed.randomSplit([0.8, 0.2])
model = pipeline.fit(train_set)
predictions = model.transform(test_set)
display(predictions)

id,delay,congestion,vehicleSpeed,distanceCovered,angle,WeekendOrNot,lineID,journeyPatternId,vehicleID,busStop,areaID,areaID2,areaID3,gridID,latitude,longitude,dateTime,partOfDay,currentHour,mean_delay,features,rawPrediction,probability,prediction
5a54d6cee45b4b563c37f8a5,147.0,0,14,0.05662943366863649,45.0,0,102,01021001,33342,6033,1163,290,1163,"164,159",53,-6,2018-01-09T14:50:40.000+0000,After-Noon,14,136.53424657534248,"List(1, 7, List(), List(1163.0, 14.0, 0.0, 147.0, 53.0, -6.0, 14.0))","List(1, 2, List(), List(18.32917297511812, 1.6708270248818802))","List(1, 2, List(), List(0.916458648755906, 0.08354135124409401))",0.0
5a9eeb1ee45b4b7708cc3f9b,505.0,0,12,0.1044455544052218,45.0,0,39,039A1001,44291,1906,4451,277,4451,"142,112",53,-6,2018-03-06T19:25:11.000+0000,Evening,19,136.53424657534248,"List(1, 7, List(), List(4451.0, 19.0, 0.0, 505.0, 53.0, -6.0, 12.0))","List(1, 2, List(), List(14.546491228070176, 5.453508771929824))","List(1, 2, List(), List(0.7273245614035088, 0.27267543859649124))",0.0
5ab0d786e45b4b101361147d,564.0,0,11,0.06554059480405244,225.0,0,49,00490001,33531,2532,2359,147,2359,"125,64",53,-6,2018-03-20T09:42:25.000+0000,Mid-Morning,9,136.53424657534248,"List(1, 7, List(), List(2359.0, 9.0, 0.0, 564.0, 53.0, -6.0, 11.0))","List(1, 2, List(), List(13.807758833703978, 6.1922411662960215))","List(1, 2, List(), List(0.6903879416851989, 0.3096120583148011))",0.0
5ae85ca6e45b4b5d974cb12b,-181.0,0,21,0.11741874248100537,45.0,0,26,066A1002,44253,1479,71676,279,4479,"153,103",53,-6,2018-05-01T12:25:05.000+0000,Noon,12,136.53424657534248,"List(1, 7, List(), List(71676.0, 12.0, 0.0, -181.0, 53.0, -6.0, 21.0))","List(1, 2, List(), List(19.96969696969697, 0.030303030303030304))","List(1, 2, List(), List(0.9984848484848484, 0.0015151515151515152))",0.0
5aec0d00f0e26f116182436c,136.53424,0,16,0.09439793136792597,225.0,0,7,007B1002,44029,776,15559,242,3889,"163,91",53,-6,2018-05-04T07:34:19.000+0000,Morning,7,136.53424657534248,"List(1, 7, List(), List(15559.0, 7.0, 0.0, 136.53424072265625, 53.0, -6.0, 16.0))","List(1, 2, List(), List(18.99763124199744, 1.0023687580025609))","List(1, 2, List(), List(0.949881562099872, 0.05011843790012804))",0.0
5b16ce0ff0e26f276439eee9,136.53424,0,23,0.15167942567170192,225.0,0,27B,027B0001,33497,6234,4544,283,4544,"155,143",53,-6,2018-06-05T17:53:13.000+0000,Evening,17,136.53424657534248,"List(1, 7, List(), List(4544.0, 17.0, 0.0, 136.53424072265625, 53.0, -6.0, 23.0))","List(1, 2, List(), List(17.635536611481754, 2.364463388518244))","List(1, 2, List(), List(0.8817768305740877, 0.1182231694259122))",0.0
596a08c1e45b4b68bea4d5be,-374.0,0,0,0.03287518011634771,45.0,0,46A,046A0001,44226,2018,4253,265,4253,"185,63",53,-6,2017-07-15T12:20:41.000+0000,Noon,12,136.53424657534248,"List(1, 7, List(), List(4253.0, 12.0, 0.0, -374.0, 53.0, -6.0, 0.0))","List(1, 2, List(), List(3.066037368204551, 16.93396263179545))","List(1, 2, List(), List(0.15330186841022755, 0.8466981315897725))",1.0
59faee85e45b4b63417ee1ad,324.0,0,29,0.16329751967709896,45.0,0,56A,056A1001,33473,2767,2362,147,2362,"122,80",53,-6,2017-11-02T10:06:59.000+0000,Mid-Morning,10,136.53424657534248,"List(1, 7, List(), List(2362.0, 10.0, 0.0, 324.0, 53.0, -6.0, 29.0))","List(1, 2, List(), List(18.465536611481753, 1.534463388518244))","List(1, 2, List(), List(0.9232768305740878, 0.07672316942591222))",0.0
5a01a7c3e45b4b356eda5427,107.0,0,3,0.019876533830114035,225.0,0,151,01510001,44266,1406,15147,236,3786,"146,96",53,-6,2017-11-07T12:31:47.000+0000,Noon,12,136.53424657534248,"List(1, 7, List(), List(15147.0, 12.0, 0.0, 107.0, 53.0, -6.0, 3.0))","List(1, 2, List(), List(18.62251461988304, 1.3774853801169589))","List(1, 2, List(), List(0.9311257309941521, 0.06887426900584795))",0.0
5a384470e45b4b179c5f3ae6,-37.0,0,32,0.1736766842345712,225.0,0,38,038A0001,33589,58,17926,279,4481,"150,111",53,-6,2017-12-18T22:42:06.000+0000,Night,22,136.53424657534248,"List(1, 7, List(), List(17926.0, 22.0, 0.0, -37.0, 53.0, -6.0, 32.0))","List(1, 2, L

In [0]:
f1 = f1_evaluator.evaluate(predictions)
recall = predictions.filter((col("congestion") == 1) & (col("prediction") == 1)).count() / predictions.filter(col("congestion") == 1).count()
print(f"f1: {f1}, recall: {recall}")

f1: 0.8274430036443765, recall: 0.8333333333333334

In [0]:
def _uncertain_delay(delay):
  if -600<delay and delay<=600:
    return 0
  else:
    return 1

uncertain_delay = udf(_uncertain_delay)

df_w_uncertain = dataset.withColumn("uncertain_delay",uncertain_delay("delay").cast("int"))\
                        .withColumn("median_delay",lit(float(median_delay))).withColumn("delay",impute_med("delay","median_delay").cast("float"))\
                        .withColumn("median_speed", lit(float(median_speed))).withColumn("speed", impute_med("speed","median_speed").cast("float"))
display(df_w_uncertain)

id,delay,congestion,vehicleSpeed,distanceCovered,angle,WeekendOrNot,lineID,journeyPatternId,vehicleID,busStop,areaID,areaID2,areaID3,gridID,latitude,longitude,dateTime,partOfDay,currentHour,uncertain_delay,median_delay
5b965216f0e26f3d4e2a418e,52.0,0,36,0.2149286706335998,225.0,0,67,00670001,44218,7078,17889,279,4472,"146,104",53,-6,2018-09-10T11:14:24.000+0000,Mid-Morning,11,0,141.0
597510dfe45b4b5d503f1f20,141.0,0,0,0.16516981395284672,45.0,0,25,025A1002,33609,4619,2935,183,2935,"101,100",53,-6,2017-07-23T21:10:29.000+0000,Night,21,0,141.0
5a79dfe1e45b4b0b521156e3,244.0,0,0,0.0020059150470843837,45.0,0,11,00111001,44241,878,3887,242,3887,"164,81",53,-6,2018-02-06T17:03:23.000+0000,Evening,17,0,141.0
5ab0d786e45b4b101361147d,564.0,0,11,0.06554059480405244,225.0,0,49,00490001,33531,2532,2359,147,2359,"125,64",53,-6,2018-03-20T09:42:25.000+0000,Mid-Morning,9,0,141.0
5b16ce0ff0e26f276439eee9,141.0,0,23,0.15167942567170192,225.0,0,27B,027B0001,33497,6234,4544,283,4544,"155,143",53,-6,2018-06-05T17:53:13.000+0000,Evening,17,1,141.0
5aec0d00f0e26f116182436c,141.0,0,16,0.09439793136792597,225.0,0,7,007B1002,44029,776,15559,242,3889,"163,91",53,-6,2018-05-04T07:34:19.000+0000,Morning,7,1,141.0
5b1ee017f0e26f6314cddbb1,-576.0,0,32,0.1911939144033726,225.0,0,7,007A0002,43099,3207,4285,267,4285,"196,65",53,-6,2018-06-11T20:48:10.000+0000,Evening,20,0,141.0
5ae85ca6e45b4b5d974cb12b,-181.0,0,21,0.11741874248100537,45.0,0,26,066A1002,44253,1479,71676,279,4479,"153,103",53,-6,2018-05-01T12:25:05.000+0000,Noon,12,0,141.0
5a9eeb1ee45b4b7708cc3f9b,505.0,0,12,0.1044455544052218,45.0,0,39,039A1001,44291,1906,4451,277,4451,"142,112",53,-6,2018-03-06T19:25:11.000+0000,Evening,19,0,141.0
5a54d6cee45b4b563c37f8a5,147.0,0,14,0.05662943366863649,45.0,0,102,01021001,33342,6033,1163,290,1163,"164,159",53,-6,2018-01-09T14:50:40.000+0000,After-Noon,14,0,141.0


In [0]:
features = ["areaID", "currentHour", "WeekendOrNot", "delay", "latitude", "longitude", "vehicleSpeed","uncertain_delay"]
va = VectorAssembler(inputCols=features, outputCol="features")
rf = RandomForestClassifier(labelCol="congestion", featuresCol="features")
pipeline = Pipeline(stages=[va, rf])
f1_evaluator = MulticlassClassificationEvaluator(labelCol="congestion", predictionCol="prediction", metricName="f1")

In [0]:
train_set, test_set = df_w_uncertain.randomSplit([0.8, 0.2])
model = pipeline.fit(train_set)
predictions = model.transform(test_set)
display(predictions)

id,delay,congestion,vehicleSpeed,distanceCovered,angle,WeekendOrNot,lineID,journeyPatternId,vehicleID,busStop,areaID,areaID2,areaID3,gridID,latitude,longitude,dateTime,partOfDay,currentHour,mean_delay,features,rawPrediction,probability,prediction
5a394627e45b4b179c6e370a,-178.0,0,0,0.0,-2.0,0,4,00041001,44150,478,979,244,979,"173,88",53,-6,2017-12-19T17:01:48.000+0000,Evening,17,136.53424657534248,"List(1, 7, List(), List(979.0, 17.0, 0.0, -178.0, 53.0, -6.0, 0.0))","List(1, 2, List(), List(7.040275513959724, 12.959724486040276))","List(1, 2, List(), List(0.3520137756979862, 0.6479862243020138))",1.0
5aec0d00f0e26f116182436c,136.53424,0,16,0.09439793136792597,225.0,0,7,007B1002,44029,776,15559,242,3889,"163,91",53,-6,2018-05-04T07:34:19.000+0000,Morning,7,136.53424657534248,"List(1, 7, List(), List(15559.0, 7.0, 0.0, 136.53424072265625, 53.0, -6.0, 16.0))","List(1, 2, List(), List(18.50996871839537, 1.49003128160463))","List(1, 2, List(), List(0.9254984359197685, 0.0745015640802315))",0.0
5a168799e45b4b356efc5407,132.0,0,23,0.3688458337564888,45.0,0,39,039A1001,36043,1698,4444,277,4444,"135,116",53,-6,2017-11-23T08:31:48.000+0000,Morning,8,136.53424657534248,"List(1, 7, List(), List(4444.0, 8.0, 0.0, 132.0, 53.0, -6.0, 23.0))","List(1, 2, List(), List(19.46996871839537, 0.5300312816046299))","List(1, 2, List(), List(0.9734984359197686, 0.026501564080231493))",0.0
5a7ac463e45b4b0b521d4aa9,136.53424,0,17,0.09039242879346472,225.0,0,16,00161001,33428,7615,73065,285,4566,"155,106",53,-6,2018-02-07T09:18:14.000+0000,Mid-Morning,9,136.53424657534248,"List(1, 7, List(), List(73065.0, 9.0, 0.0, 136.53424072265625, 53.0, -6.0, 17.0))","List(1, 2, List(), List(17.836179544606196, 2.163820455393804))","List(1, 2, List(), List(0.8918089772303098, 0.1081910227696902))",0.0
5aa953cde45b4b4b555bfcc6,-139.0,0,53,0.2524470106865519,225.0,0,67,067X0004,44299,312,73050,285,4565,"155,103",53,-6,2018-03-14T16:54:28.000+0000,Evening,16,136.53424657534248,"List(1, 7, List(), List(73050.0, 16.0, 0.0, -139.0, 53.0, -6.0, 53.0))","List(1, 2, List(), List(19.905623989050643, 0.09437601094935921))","List(1, 2, List(), List(0.9952811994525321, 0.00471880054746796))",0.0
5a8c7b8fe45b4b77081b48ea,426.0,0,38,0.21230361745758622,45.0,0,18,00180001,33646,2790,3881,242,3881,"157,91",53,-6,2018-02-20T19:48:24.000+0000,Evening,19,136.53424657534248,"List(1, 7, List(), List(3881.0, 19.0, 0.0, 426.0, 53.0, -6.0, 38.0))","List(1, 2, List(), List(18.69124531414005, 1.308754685859949))","List(1, 2, List(), List(0.9345622657070025, 0.06543773429299746))",0.0
5a8d32d6e45b4b770821bfeb,116.0,0,26,0.1450596869576156,45.0,0,75,00750002,36033,453,3905,243,3905,"174,71",53,-6,2018-02-21T08:50:25.000+0000,Morning,8,136.53424657534248,"List(1, 7, List(), List(3905.0, 8.0, 0.0, 116.0, 53.0, -6.0, 26.0))","List(1, 2, List(), List(18.454413162839813, 1.5455868371601853))","List(1, 2, List(), List(0.9227206581419907, 0.07727934185800926))",0.0
597c70a9e45b4b23a99fa5c3,228.0,0,0,0.12259941828628719,45.0,0,27,077A1001,43111,2382,17909,279,4477,"150,99",53,-6,2017-07-29T11:25:10.000+0000,Mid-Morning,11,136.53424657534248,"List(1, 7, List(), List(17909.0, 11.0, 0.0, 228.0, 53.0, -6.0, 0.0))","List(1, 2, List(), List(5.574422917244122, 14.42557708275588))","List(1, 2, List(), List(0.2787211458622061, 0.7212788541377939))",1.0
59f227b8e45b4b6341222518,579.0,0,24,0.11654264694294317,225.0,0,18,00181001,33347,2206,3039,189,3039,"119,102",53,-6,2017-10-26T18:20:51.000+0000,Evening,18,136.53424657534248,"List(1, 7, List(), List(3039.0, 18.0, 0.0, 579.0, 53.0, -6.0, 24.0))","List(1, 2, List(), List(17.72928879240092, 2.2707112075990796))","List(1, 2, List(), List(0.886464439620046, 0.11353556037995398))",0.0
5a745c62e45b4b35fc84c414,136.53424,0,0,0.0,-2.0,0,39,039A0001,36061,1664,4444,277,4444,"137,115",53,-6,2018-02-02T12:40:32.000+0000,Noon,12,136.53424657534248,"List(1, 7, List(), List(4444.0, 12.0, 0.0, 136.53424072265625, 53.0, -6.0, 0.0))","List(1, 2, List(), List(6.404472609793815, 13.

In [0]:
f1 = f1_evaluator.evaluate(predictions)
recall = predictions.filter((col("congestion") == 1) & (col("prediction") == 1)).count() / predictions.filter(col("congestion") == 1).count()
print(f"f1: {f1}, recall: {recall}")

f1: 0.8246700188560654, recall: 0.8571428571428571

In [0]:
features = ["areaID", "currentHour", "WeekendOrNot", "delay", "latitude", "longitude", "vehicleSpeed"]
va = VectorAssembler(inputCols=features, outputCol="features")
rf = RandomForestClassifier(labelCol="congestion", featuresCol="features")
pipeline = Pipeline(stages=[va, rf])
f1_evaluator = MulticlassClassificationEvaluator(labelCol="congestion", predictionCol="prediction", metricName="f1")

In [0]:
train_set, test_set = df_w_uncertain.randomSplit([0.8, 0.2])
model = pipeline.fit(train_set)
predictions = model.transform(test_set)
display(predictions)

id,delay,congestion,vehicleSpeed,distanceCovered,angle,WeekendOrNot,lineID,journeyPatternId,vehicleID,busStop,areaID,areaID2,areaID3,gridID,latitude,longitude,dateTime,partOfDay,currentHour,uncertain_delay,median_delay,features,rawPrediction,probability,prediction
5a699d0ee45b4b35fce71b1a,-175.0,0,17,0.14650574696053137,225.0,0,27,00270004,43111,1220,4621,288,4621,"173,122",53,-6,2018-01-25T09:01:47.000+0000,Mid-Morning,9,0,141.0,"List(1, 7, List(), List(4621.0, 9.0, 0.0, -175.0, 53.0, -6.0, 17.0))","List(1, 2, List(), List(19.07728271908314, 0.922717280916858))","List(1, 2, List(), List(0.9538641359541572, 0.04613586404584291))",0.0
5ae85ca6e45b4b5d974cb12b,-181.0,0,21,0.11741874248100537,45.0,0,26,066A1002,44253,1479,71676,279,4479,"153,103",53,-6,2018-05-01T12:25:05.000+0000,Noon,12,0,141.0,"List(1, 7, List(), List(71676.0, 12.0, 0.0, -181.0, 53.0, -6.0, 21.0))","List(1, 2, List(), List(19.358603049165662, 0.6413969508343373))","List(1, 2, List(), List(0.9679301524582831, 0.032069847541716864))",0.0
596a08c1e45b4b68bea4d5be,-374.0,0,0,0.03287518011634771,45.0,0,46A,046A0001,44226,2018,4253,265,4253,"185,63",53,-6,2017-07-15T12:20:41.000+0000,Noon,12,0,141.0,"List(1, 7, List(), List(4253.0, 12.0, 0.0, -374.0, 53.0, -6.0, 0.0))","List(1, 2, List(), List(4.868871807243668, 15.13112819275633))","List(1, 2, List(), List(0.24344359036218338, 0.7565564096378166))",1.0
5aa0f0a6e45b4b7708e7ccc4,591.0,0,17,0.10879024089830092,45.0,0,37,00371001,44047,1689,3079,192,3079,"123,118",53,-6,2018-03-08T08:13:18.000+0000,Morning,8,0,141.0,"List(1, 7, List(), List(3079.0, 8.0, 0.0, 591.0, 53.0, -6.0, 17.0))","List(1, 2, List(), List(16.899467661401566, 3.100532338598434))","List(1, 2, List(), List(0.8449733830700783, 0.1550266169299217))",0.0
5a65beeae45b4b35fcaa5f71,-505.0,0,1,0.007450060085337941,0.0,0,56A,056A1001,43073,340,73058,285,4565,"157,103",53,-6,2018-01-22T10:37:16.000+0000,Mid-Morning,10,0,141.0,"List(1, 7, List(), List(73058.0, 10.0, 0.0, -505.0, 53.0, -6.0, 1.0))","List(1, 2, List(), List(16.164305966937548, 3.835694033062454))","List(1, 2, List(), List(0.8082152983468773, 0.1917847016531227))",0.0
5960b490e45b4b68be2e7c2d,0.0,0,0,0.00597669742665818,45.0,1,75,00751001,33641,2038,68642,267,4289,"198,75",53,-6,2017-07-08T10:31:14.000+0000,Mid-Morning,10,0,141.0,"List(1, 7, List(), List(68642.0, 10.0, 1.0, 0.0, 53.0, -6.0, 0.0))","List(1, 2, List(), List(10.492305109178174, 9.507694890821826))","List(1, 2, List(), List(0.5246152554589087, 0.4753847445410913))",0.0
59f9929de45b4b63416c6bbd,-333.0,0,43,0.24368681681266277,45.0,0,13,00131004,43049,6238,4500,280,4500,"152,134",53,-6,2017-11-01T09:22:33.000+0000,Mid-Morning,9,0,141.0,"List(1, 7, List(), List(4500.0, 9.0, 0.0, -333.0, 53.0, -6.0, 43.0))","List(1, 2, List(), List(19.07728271908314, 0.922717280916858))","List(1, 2, List(), List(0.9538641359541572, 0.04613586404584291))",0.0
5a72e9b1e45b4b35fc6d0231,146.0,0,15,0.09265448066306363,225.0,0,25,025B0002,38056,1445,71650,279,4477,"150,103",53,-6,2018-02-01T10:19:00.000+0000,Mid-Morning,10,0,141.0,"List(1, 7, List(), List(71650.0, 10.0, 0.0, 146.0, 53.0, -6.0, 15.0))","List(1, 2, List(), List(17.76601045657307, 2.23398954342693))","List(1, 2, List(), List(0.8883005228286536, 0.1116994771713465))",0.0
5b2fa36df0e26f6314b866b8,150.0,0,26,0.15222322442519512,225.0,1,46A,046A1001,44111,776,15559,242,3889,"163,91",53,-6,2018-06-24T13:57:49.000+0000,Noon,13,0,141.0,"List(1, 7, List(), List(15559.0, 13.0, 1.0, 150.0, 53.0, -6.0, 26.0))","List(1, 2, List(), List(17.227413965345, 2.7725860346549998))","List(1, 2, List(), List(0.86137069826725, 0.13862930173274998))",0.0
5b59b21bf0e26f4b8bf69739,214.0,0,0,0.0,-2.0,0,42,00420002,38059,3575,313,313,313,"184,146",53,-6,2018-07-26T11:35:47.000+0000,Mid-Morning,11,0,141.0,"List(1, 7, List(), List(313.0, 11.0, 0.0, 214.0, 53.0, -6.0, 0.0))","List(1, 2, List(), List(2.4423171241100374, 17.557682875889963))","List(1, 2, List(), List(0.12211585620550187, 0.8778841437944982))",1.0


In [0]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(labelCol="congestion", featuresCol="features")
lr_pipeline = Pipeline(stages=[va, lr])
model = lr_pipeline.fit(train_set_small)
predictions = model.transform(test_set_small)
f1 = f1_evaluator.evaluate(predictions)
recall = predictions.filter((col("congestion") == 1) & (col("prediction") == 1)).count() / predictions.filter(col("congestion") == 1).count()
print(f"f1: {f1}, recall:{recall}")

f1: 0.8973009446693658, recall:0.9473684210526315

In [0]:
df : f1: 0.7871794871794873, recall:0.9523809523809523
df_filtered: f1: 0.8973009446693658, recall:0.9473684210526315
med: f1: 0.8973009446693658, recall:0.9473684210526315
mean: f1: 0.7871794871794873, recall:0.9473684210526315
med_w_uncert: f1: 0.8973009446693658, recall:0.9473684210526315